# Demo: `DAOD_PHYSLITE` analysis with uproot/awkward on jupyterhub on GCP

<div class="alert alert-info">
Note: This tutorial is targeted at users interested in R&D and technical details. Much of this is still in early development/prototyping.
</div>

## Read and process PHYSLITE using uproot/awkward

First, let's start with some general notes on reading `DAOD_PHYSLITE`

The PHYSLITE ROOT files currently follow a similar structure as regular ATLAS xAODs

They containing several trees, where the one holding the actual data is called `CollectionTree`. The others contain various forms of Metadata.

In [1]:
import uproot
import awkward as ak

In [2]:
f = uproot.open("data/DAOD_PHYSLITE_21.2.108.0.art.pool.root")

In [3]:
f.keys()

['MetaData;2',
 'MetaData;1',
 'MetaDataHdr;2',
 'MetaDataHdr;1',
 'MetaDataHdrForm;2',
 'MetaDataHdrForm;1',
 'POOLContainer;2',
 'POOLContainer;1',
 'POOLContainerForm;2',
 'POOLContainerForm;1',
 '##Params;2',
 '##Params;1',
 '##Shapes;2',
 '##Shapes;1',
 '##Links;2',
 '##Links;1',
 'CollectionTree;1']

### 1-D vectors
* All branches are stored with the **highest split level**
* In most cases data stored in branches called `Aux.<something>` or `AuxDyn.<something>`
* Typically **vectors of fundamental types**, like e.g. pt/eta/phi of particle collections
* **can be read into numpy arrays efficiently using uproot** since data stored as contiguous blocks  
(except for the 10-byte vector headers whoose positions are known from ROOT's event offsets)

In [4]:
f["CollectionTree"].show("/AnalysisElectronsAuxDyn.(pt|eta|phi)$/i", name_width=30, interpretation_width=50)

name                           | typename                 | interpretation                                    
-------------------------------+--------------------------+---------------------------------------------------
AnalysisElectronsAuxDyn.pt     | std::vector<float>       | AsJagged(AsDtype('>f4'), header_bytes=10)
AnalysisElectronsAuxDyn.eta    | std::vector<float>       | AsJagged(AsDtype('>f4'), header_bytes=10)
AnalysisElectronsAuxDyn.phi    | std::vector<float>       | AsJagged(AsDtype('>f4'), header_bytes=10)


### ElementLinks

The most relevant exception to this: `ElementLink` branches:

* provide cross references into other collections
* **often 2-dimensional** (`vector<vector<ElementLink<...>>>`)
* data part (`ElementLink`) is serialized as a **structure of 2 32bit unsigned integers**:
  * hash `m_persKey`, identifying the target collection
  * index `m_persIndex` identifying the array-index of the corresponding particle in the target collection.

In [5]:
f["CollectionTree/AnalysisElectronsAuxDyn.trackParticleLinks"].typename

'std::vector<std::vector<ElementLink<DataVector<xAOD::TrackParticle_v1>>>>'

In [6]:
for element in f.file.streamer_named("ElementLinkBase").elements:
    print(f"{element.member('fName')}: {element.member('fTypeName')}")

m_persKey: unsigned int
m_persIndex: unsigned int


Uproot can read this, but the loop that deserializes the data is done in python and therefore slow.

This is not relevant for this very small file, but becomes important for larger files.

This can be handled by [AwkwardForth](https://doi.org/10.1051/epjconf/202125103002) which is however currently (November 2021) not yet integrated with uproot.

For now we can use a custom function `branch_to_array` to do this:

In [7]:
from physlite_experiments.deserialization_hacks import branch_to_array

In [8]:
branch_to_array(f["CollectionTree/AnalysisElectronsAuxDyn.trackParticleLinks"])

<Array [[], [], ... m_persIndex: 1}]]] type='40 * var * var * {"m_persKey": int3...'>

One can actually see a significant improvement already for the small file with only 40 events!

In [9]:
%%timeit
# using standard uproot
f.file.array_cache.clear()
f["CollectionTree/AnalysisElectronsAuxDyn.trackParticleLinks"].array()

7.86 ms ± 281 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
%%timeit
# using numba
f.file.array_cache.clear()
branch_to_array(f["CollectionTree/AnalysisElectronsAuxDyn.trackParticleLinks"])

772 µs ± 18.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [11]:
%%timeit
# using awkward forth
f.file.array_cache.clear()
branch_to_array(f["CollectionTree/AnalysisElectronsAuxDyn.trackParticleLinks"], use_forth=True)

809 µs ± 4.99 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Integration with `coffea.nanoevents`

The PHYSLITE schema and the corresponding behavior classes are still under development - [CoffeaTeam/coffea#540](https://github.com/CoffeaTeam/coffea/issues/540) tracks the progress of some TODO items.

For more information on `NanoEvents` see the [NanoEvents tutorial](https://github.com/CoffeaTeam/coffea/blob/master/binder/nanoevents.ipynb) or [Nick Smith's presentation](https://youtu.be/udzkE6t4Mck) at the [pyHEP 2020](https://indico.cern.ch/event/882824).

<div class="alert alert-block alert-success">
    <b>The Goal:</b>
    <ul>
        <li>Work with object-oriented event data models, but stick to the array-at-a-time processing paradigm.<br> → Struct/Object of arrays instead of Array of structs/objects</li>
        <li>Hide the details from the user</li>
    </ul>
</div>

In [12]:
from coffea.nanoevents import NanoEventsFactory, PHYSLITESchema

# patch nanoevents to use the custom branch_to_array function
from physlite_experiments.deserialization_hacks import patch_nanoevents
patch_nanoevents()

In [13]:
factory = NanoEventsFactory.from_root(
    "data/DAOD_PHYSLITE_21.2.108.0.art.pool.root",
    "CollectionTree",
    schemaclass=PHYSLITESchema
)
events = factory.events()

/srv/conda/envs/notebook/lib/python3.8/site-packages/coffea/nanoevents/mapping/uproot.py:96: UserWarning: Skipping McEventInfo as it is it cannot be represented as an Awkward array
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.8/site-packages/coffea/nanoevents/mapping/uproot.py:91: UserWarning: Skipping EventInfoAux./EventInfoAux.xAOD::AuxInfoBase as it is not interpretable by Uproot
  warnings.warn(f"Skipping {key} as it is not interpretable by Uproot")
/srv/conda/envs/notebook/lib/python3.8/site-packages/coffea/nanoevents/mapping/uproot.py:91: UserWarning: Skipping xTrigDecisionAux./xTrigDecisionAux.xAOD::AuxInfoBase as it is not interpretable by Uproot
  warnings.warn(f"Skipping {key} as it is not interpretable by Uproot")
/srv/conda/envs/notebook/lib/python3.8/site-packages/coffea/nanoevents/mapping/uproot.py:91: UserWarning: Skipping METAssoc_AnalysisMETAux./METAssoc_AnalysisMETAux.xAOD::AuxContainerBase as it is not interpretable by Uproot
  warnings.warn(f"Skipping {key}

This groups particles and the available properties conveniently under one central `event` array

* everything is lazy loading
* cross referencing via ElementLinks already implemented for some collections
* particles behave as LorentzVectors (can add them, calculate invariant masses and much more)

See [my tutorial at the IRIS-HEP AGC tools workshop 2021](https://github.com/nikoladze/agc-tools-workshop-2021-physlite) for more technical details

In [14]:
events.Electrons

<ElectronArray [[], [], ... Electron], [Electron]] type='40 * var * electron'>

In [15]:
events.Electrons.fields

['trackParticleLinks',
 'pt',
 '_eventindex',
 'eta',
 'phi',
 'm',
 'charge',
 'ptvarcone30_TightTTVA_pt1000',
 'ptcone20_TightTTVA_pt1000',
 'ptvarcone20_TightTTVA_pt1000',
 'ptvarcone30_TightTTVA_pt500',
 'ptcone20_TightTTVA_pt500',
 'DFCommonElectronsLHLoose',
 'topoetcone20',
 'ptvarcone20',
 'truthOrigin',
 'truthParticleLink.m_persKey',
 'truthParticleLink.m_persIndex',
 'truthType',
 'author',
 'ptvarcone40',
 'caloClusterLinks',
 'OQ',
 'ambiguityLink.m_persKey',
 'ambiguityLink.m_persIndex',
 'ambiguityType',
 'topoetcone20ptCorrection',
 'DFCommonElectronsLHVeryLoose',
 'DFCommonElectronsLHVeryLooseIsEMValue',
 'DFCommonElectronsLHLooseIsEMValue',
 'DFCommonElectronsLHLooseBL',
 'DFCommonElectronsLHLooseBLIsEMValue',
 'DFCommonElectronsLHMedium',
 'DFCommonElectronsLHMediumIsEMValue',
 'DFCommonElectronsLHTight',
 'DFCommonElectronsLHTightIsEMValue',
 'DFCommonElectronsECIDS',
 'DFCommonElectronsECIDSResult',
 'truthPdgId',
 'firstEgMotherTruthType',
 'firstEgMotherTruthOrig

In [16]:
events.Electrons.trackParticles

<TrackParticleArray [[], [], ... TrackParticle]]] type='40 * var * var * ?trackP...'>

In [17]:
events.Electrons.trackParticles.z0

<Array [[], [], ... [[-8.73, -0.0982]]] type='40 * var * var * ?float32[paramete...'>

In [18]:
events.Electrons[events.Electrons.pt > 10000].trackParticles

<TrackParticleArray [[], [], ... TrackParticle]]] type='40 * var * var * ?trackP...'>

In [19]:
events.TruthElectrons.parents

<TruthParticleArray [[[TruthParticle], ... TruthParticle]]] type='40 * truthPart...'>

In [20]:
events.TruthElectrons.parents.children

<TruthParticleArray [[[[TruthParticle, ... TruthParticle]]]] type='40 * truthPar...'>

In [21]:
events.TruthElectrons.parents.children.parents

<TruthParticleArray [[[[[TruthParticle, ... ] type='40 * truthParticle'>

In [22]:
events.TruthElectrons.parents.children.parents.children.pdgId

<Array [[[[[[16, 11, -12, ... -12, 22, 22]]]]]] type='40 * var * var * option[va...'>

In [23]:
events.TruthElectrons.parents.children.parents.children.pdgId.ndim

6

## Read data via HTTPS from google cloud storage (authentication via rucio)

*Now we are going to do something a bit weird: instead of importing some utility functions we will directly execute a python file containing them. This is because we later want dask to serialize the functions to send them to the workers (which don't have access to our local directory on the submission node). It's a workaround for interactively developing functions that are sent to dask workers on a dask gateway cluster (which is used here). This issue does not occur in a setting where you have a shared filesystem for all workers.*

**Let me know if you know a better approach - one alternative is dask's `upload_file` method, but that has it's own issues**

**Note: trying the `UploadFile` worker plugin now, see https://distributed.dask.org/en/latest/plugins.html#built-in-worker-plugins**

In [24]:
from utils import setup_rucio_and_proxy, get_signed_url, get_signed_url_worker

We will use these functions to authenticate to rucio and get signed urls on google cloud storage (GCS).

For that we have to provide a VOMS proxy. To avoid the need for having the grid certificate and the voms tools on this jupyterhub instance we create the voms proxy outside (some machine where we have the voms tools and our grid certificate) and upload it to this notebook:

In [25]:
from ipywidgets import FileUpload
upload = FileUpload()
display(upload)

FileUpload(value={}, description='Upload')

Then we setup the nescessary environment variables (fill in your cern account name):

In [26]:
setup_rucio_and_proxy(upload.data[-1], rucio_account="nihartma")

Now we should be able to query rucio:

In [27]:
import rucio.client
rucio_client = rucio.client.Client(ca_cert=False)

/srv/conda/envs/notebook/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'voatlasrucio-auth-prod.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


Let's get a list of all files in one data period, corresponding to around 10% of the whole Run2 data - around 10TB in total:

In [28]:
files = list(rucio_client.list_files("data17_13TeV", "data17_13TeV.periodK.physics_Main.PhysCont.DAOD_PHYSLITE.grp17_v01_p4309"))

/srv/conda/envs/notebook/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'voatlasrucio-server-prod.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [29]:
files[0]

{'scope': 'data17_13TeV',
 'name': 'DAOD_PHYSLITE.22958312._000001.pool.root.1',
 'bytes': 339098731,
 'adler32': 'ea4a58e1',
 'guid': '9182E93759873A4BA6ABC72E1C286873',
 'events': 42870}

In [30]:
sum(file["bytes"] for file in files) / 1024 ** 4

10.121311471758418

The full Run2 dataset is replicated to GCS. To access it via https we can ask rucio for a signed url. Uproot can directly deal with http(s) urls:

In [31]:
url = get_signed_url(rucio_client, files[0]["scope"], files[0]["name"])

/srv/conda/envs/notebook/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'voatlasrucio-server-prod.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [32]:
f_remote = uproot.open(url)

In [33]:
f_remote["CollectionTree/AnalysisElectronsAuxDyn.pt"].array()

<Array [[], [], [], [], ... [], [4.81e+03], []] type='42870 * var * float32'>

Some notes on this:

* GCS does not support multi-range requests (equivalent to xrootd vector reads), single-range requests are allowed
* Single-range requests with the uproot `MultithreadedHTTPSource` are suboptimal
* GCS seems fine with a huge number of parallel requests - this can be done with asyncio
* However, oftentimes downloading the whole file is still faster async reading of partial chunks (but needs lot's of memory)

In [34]:
import requests

def download(url):
    return requests.get(url).content

In [35]:
data = download(url)

In [36]:
import io

uproot.open(io.BytesIO(data))["CollectionTree/AnalysisElectronsAuxDyn.pt"].array()

<Array [[], [], [], [], ... [], [4.81e+03], []] type='42870 * var * float32'>

I have an experimental implementation for an asyncio HTTPSource for uproot (should probably make a PR for uproot at some point or consider using an interface to fsspec which has a `cat_ranges` method that might be used for this).

GCS seems fine with 100 parallel tcp connections (even for each worker on a larger cluster):

In [37]:
from physlite_experiments.io import AIOHTTPSource

class AIOHTTP100Source(AIOHTTPSource):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, tcp_connection_limit=100, **kwargs)

In [38]:
uproot.open(url, http_handler=AIOHTTP100Source)["CollectionTree/AnalysisElectronsAuxDyn.pt"].array()

<Array [[], [], [], [], ... [], [4.81e+03], []] type='42870 * var * float32'>

## Run an actual analysis with this

In [51]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    factory = NanoEventsFactory.from_root(url, "CollectionTree", schemaclass=PHYSLITESchema, uproot_options=dict(http_handler=AIOHTTP100Source))

In [52]:
events = factory.events()

In [53]:
events.Electrons.trackParticles.z0

<Array [[], [], [], [], ... [[19.8, 24.8]], []] type='42870 * var * var * ?float...'>

In [54]:
from physlite_experiments.analysis_example import get_obj_sel

In [55]:
def run_analysis(events):
    events = get_obj_sel(events)
    return  {
        collection: {
            flag : ak.count_nonzero(events[collection][flag])
            for flag in ["baseline", "passOR", "signal"]
        } for collection in ["Electrons", "Muons", "Jets"]
    }

In [56]:
def merge(results):
    out = {
        collection: {
            flag: 0
            for flag in ["baseline", "passOR", "signal"]
        } for collection in ["Electrons", "Muons", "Jets"]
    }
    for result in results:
        for collection, flags in result.items():
            for flag, count in flags.items():
                out[collection][flag] += count
    return out

In [57]:
%%time
result = run_analysis(events)

/srv/conda/envs/notebook/lib/python3.8/site-packages/aiohttp/client.py:331: ResourceWarning: Unclosed client session <aiohttp.client.ClientSession object at 0x7f539f373ee0>
  _warnings.warn(
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f539f373ee0>
/srv/conda/envs/notebook/lib/python3.8/site-packages/aiohttp/client.py:331: ResourceWarning: Unclosed client session <aiohttp.client.ClientSession object at 0x7f539c238a90>
  _warnings.warn(
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f539c238a90>
/srv/conda/envs/notebook/lib/python3.8/site-packages/awkward/_connect/_numpy.py:195: RuntimeWarning: divide by zero encountered in true_divide
  result = getattr(ufunc, method)(


CPU times: user 6.37 s, sys: 623 ms, total: 6.99 s
Wall time: 9.46 s


In [58]:
result

{'Electrons': {'baseline': 5417, 'passOR': 5373, 'signal': 4005},
 'Muons': {'baseline': 9740, 'passOR': 7248, 'signal': 5264},
 'Jets': {'baseline': 260703, 'passOR': 253124, 'signal': 224775}}

## Run on a dask cluster

also see Fernandos instructions on https://github.com/gcp4hep/analysis-cluster/wiki/Daskhub-usage

In [59]:
import warnings

In [60]:
x509_data = upload.data[-1] # need to extract this such that it can be pickled

In [61]:
def job(fileinfo, max_chunksize=50000):
    patch_nanoevents()
    url = get_signed_url_worker(x509_data, fileinfo["scope"], fileinfo["name"], rucio_account="nihartma", ca_cert=False)
    results = []
    for start in range(0, fileinfo["events"], max_chunksize):
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            # note: for xrootd need a closing routine here
            with uproot.open(url, http_handler=AIOHTTP100Source) as f:
                num_entries = f["CollectionTree"].num_entries
                for start in range(0, num_entries, max_chunksize):
                    with warnings.catch_warnings():
                        warnings.simplefilter("ignore")
                        factory = NanoEventsFactory.from_root(
                            f,
                            "CollectionTree",
                            schemaclass=PHYSLITESchema,
                            entry_start=start,
                            entry_stop=start + max_chunksize
                        )
                        events = factory.events()
                    results.append(run_analysis(events))
    return merge(results)

In [62]:
job(files[0])

/srv/conda/envs/notebook/lib/python3.8/site-packages/rucio/client/baseclient.py:966: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/jovyan/.rucio_jovyan/auth_token_for_account_nihartma' mode='r' encoding='UTF-8'>
  if not self.__read_token():
/srv/conda/envs/notebook/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'voatlasrucio-server-prod.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f53ec2d8a30>


{'Electrons': {'baseline': 5417, 'passOR': 5373, 'signal': 4005},
 'Muons': {'baseline': 9740, 'passOR': 7248, 'signal': 5264},
 'Jets': {'baseline': 260703, 'passOR': 253124, 'signal': 224775}}

In [82]:
from dask_gateway import GatewayCluster
cluster = GatewayCluster(
    worker_cores=1,
    worker_memory=3.5,
    image="eu.gcr.io/gke-dev-311213/jupyter-physlite:prod"
)

In [84]:
cluster

In [72]:
#cluster.shutdown()

In [92]:
client = cluster.get_client()

In [93]:
client

Client Scheduler: gateway://traefik-dhub-dask-gateway.default:80/default.8b38658347d04221a5d8b39ed6897a22 Dashboard: /services/dask-gateway/clusters/default.8b38658347d04221a5d8b39ed6897a22/status,Cluster Workers: 8 Cores: 8 Memory: 28.00 GiB


In [87]:
cluster.scale(8)

In [88]:
#cluster.shutdown()

In [95]:
from distributed.diagnostics.plugin import UploadFile

client.register_worker_plugin(UploadFile("utils.py"))

{'tls://10.64.1.2:35937': {'status': 'repeat'},
 'tls://10.64.22.2:36015': {'status': 'repeat'},
 'tls://10.64.22.3:36041': {'status': 'repeat'},
 'tls://10.64.22.4:38403': {'status': 'repeat'},
 'tls://10.64.22.5:44027': {'status': 'repeat'},
 'tls://10.64.22.6:38733': {'status': 'repeat'},
 'tls://10.64.22.7:32855': {'status': 'repeat'},
 'tls://10.64.22.8:42067': {'status': 'repeat'}}

In [98]:
fut = client.submit(job, files[1])

In [99]:
fut.result()

{'Electrons': {'baseline': 4995, 'passOR': 4943, 'signal': 3693},
 'Muons': {'baseline': 8935, 'passOR': 6629, 'signal': 4884},
 'Jets': {'baseline': 249767, 'passOR': 242808, 'signal': 216391}}

In [100]:
# https://coiled.io/blog/tackling-unmanaged-memory-with-dask/

In [101]:
import gc

In [102]:
client.run(gc.collect)

{'tls://10.64.1.2:36895': 111,
 'tls://10.64.22.2:37747': 895,
 'tls://10.64.22.3:46513': 100,
 'tls://10.64.22.4:35023': 135,
 'tls://10.64.22.5:33533': 52,
 'tls://10.64.22.6:33743': 52,
 'tls://10.64.22.7:41723': 52,
 'tls://10.64.22.8:44539': 52}

In [103]:
import ctypes
def trim_memory() -> int:
    libc = ctypes.CDLL("libc.so.6")
    return libc.malloc_trim(0)
client.run(trim_memory)

{'tls://10.64.1.2:36895': 1,
 'tls://10.64.22.2:37747': 1,
 'tls://10.64.22.3:46513': 1,
 'tls://10.64.22.4:35023': 1,
 'tls://10.64.22.5:33533': 1,
 'tls://10.64.22.6:33743': 1,
 'tls://10.64.22.7:41723': 1,
 'tls://10.64.22.8:44539': 1}

In [105]:
futures = client.map(job, files[::100][:40])

In [104]:
futures[0].result()

{'Electrons': {'baseline': 5417, 'passOR': 5373, 'signal': 4005},
 'Muons': {'baseline': 9740, 'passOR': 7248, 'signal': 5264},
 'Jets': {'baseline': 260703, 'passOR': 253124, 'signal': 224775}}